In [10]:
%pip install pyldpc

Note: you may need to restart the kernel to use updated packages.


In [13]:
# pipeline.py — End-to-End FSO-MDM OAM Simulation Pipeline (robust build)
# Based on: Andrews & Phillips (2005), Goodman (2005), Wang et al. (Nat. Photon. 2012)
# Integrates: encoding.py, turbulence.py, fsplAttenuation.py, lgBeam.py, receiver.py
# Supports multi-ensemble BER, Strehl, scintillation & crosstalk statistics.

import os
import sys
import argparse
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, Any
import warnings
from datetime import datetime

# --------------------------------------------
# Ensure local modules importable
# --------------------------------------------
try:
    SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    SCRIPT_DIR = os.getcwd()
sys.path.insert(0, SCRIPT_DIR)

# --------------------------------------------
# Imports
# --------------------------------------------
try:
    from encoding import encodingRunner
except ImportError as e:
    raise ImportError(f"Requires encoding.py: {e}")

try:
    from turbulence import (
        AtmosphericTurbulence,
        create_multi_layer_screens,
        apply_multi_layer_turbulence,
        validate_turbulence_implementation,
        analyze_turbulence_effects,
    )
except ImportError as e:
    raise ImportError(f"Requires turbulence.py: {e}")

try:
    from fsplAttenuation import calculate_path_loss, calculate_geometric_loss
except ImportError as e:
    warnings.warn(f"fsplAttenuation.py optional (geo/atm loss fallback unity): {e}")
    def calculate_path_loss(*a, **k): return 0.0
    def calculate_geometric_loss(*a, **k): return (0.0, 1.0)

try:
    from receiver import FSORx
except ImportError as e:
    raise ImportError(f"Requires receiver.py: {e}")

try:
    from lgBeam import LaguerreGaussianBeam
except ImportError:
    LaguerreGaussianBeam = None
    warnings.warn("lgBeam optional (uses Gaussian fallback)")

warnings.filterwarnings("ignore")
np.random.seed(42)

# ===========================================================
# CONFIG
# ===========================================================
def get_default_config():
    return {
        'spatial_modes': [(0, 1), (0, -1)],
        'wavelength': 1550e-9,
        'w0': 25e-3,
        'distance': 1200.0,
        'P_tx_watts': 1.0,
        'n_ensembles': 3,
        'n_info_bits': 16384,
        'pilot_ratio': 0.05,
        'fec_rate': 0.8,
        'symbol_time_s': 1e-9,
        'receiver_radius': 0.15,
        'ground_Cn2': 1e-15,
        'cn2_model': 'uniform',
        'L0': 10.0,
        'l0': 0.005,
        'num_screens': 25,
        'oversampling': 1,
        'N_grid': 256,
        'eq_method': 'auto',
        'generate_3d': False,
        'verbose': True,
        'plot': True,
        'save_plots': True,
    }

parser = argparse.ArgumentParser(description="FSO-OAM Pipeline Simulation")
parser.add_argument('--ensembles', type=int, default=3)
parser.add_argument('--distance', type=float, default=1200.0)
parser.add_argument('--cn2', type=float, default=1e-15)
parser.add_argument('--n_bits', type=int, default=16384)
parser.add_argument('--zf', action='store_true')
args, unknown = parser.parse_known_args()
config = get_default_config()
config.update(vars(args))

# ===========================================================
# MAIN PIPELINE
# ===========================================================
def run_pipeline(config: Dict[str, Any]) -> Dict[str, Any]:
    if config['verbose']:
        print("\n" + "="*80)
        print("FSO-OAM Pipeline Simulation (Literature-Aligned)")
        print("="*80)

    start_time = datetime.now()

    # --- 1. TRANSMITTER ---
    if config['verbose']:
        print("\n[1] Transmitter Initialization")
    tx = encodingRunner(
        spatial_modes=config['spatial_modes'],
        wavelength=config['wavelength'],
        w0=config['w0'],
        fec_rate=config['fec_rate'],
        pilot_ratio=config['pilot_ratio'],
        symbol_time_s=config['symbol_time_s'],
        P_tx_watts=config['P_tx_watts']
    )
    data_bits = np.random.randint(0, 2, config['n_info_bits'])
    frame = tx.transmit(data_bits, generate_3d_field=config['generate_3d'], verbose=config['verbose'])
    tx_signals = frame.tx_signals

    symbol_lengths = [len(s['symbols']) for s in tx_signals.values()]
    if len(symbol_lengths) == 0 or min(symbol_lengths) == 0:
        raise RuntimeError("Transmitter produced no symbols.")
    n_symbols = int(min(symbol_lengths))
    n_modes = len(config['spatial_modes'])

    if config['verbose']:
        if hasattr(tx, 'ldpc'):
            print(f"   LDPC: n={tx.ldpc.n}, k={tx.ldpc.k}, rate={tx.ldpc.rate}")
        print(f"   Modes: {config['spatial_modes']} (n_modes={n_modes})")
        print(f"   Frame: {n_symbols} symbols/mode, pilots ~{int(config['pilot_ratio']*n_symbols)}")

    # --- 2. TURBULENCE ---
    if config['verbose']:
        print("\n[2] Turbulence Setup")
    beam_example = LaguerreGaussianBeam(0, 1, config['wavelength'], config['w0']) if LaguerreGaussianBeam else None
    turb = AtmosphericTurbulence(
        Cn2=config['ground_Cn2'], L0=config['L0'], l0=config['l0'],
        wavelength=config['wavelength'], w0=config['w0'], beam=beam_example
    )
    total_r0 = turb.fried_parameter(config['distance'])
    sigma_R2 = turb.rytov_variance(config['distance'], 'gaussian')
    layers = create_multi_layer_screens(
        config['distance'], config['num_screens'], config['wavelength'],
        ground_Cn2=config['ground_Cn2'], L0=config['L0'], l0=config['l0'],
        cn2_model=config['cn2_model'], verbose=config['verbose']
    )
    validate_turbulence_implementation()
    if config['verbose']:
        print(f"   Path: {config['distance']} m, Cn²={config['ground_Cn2']:.1e}, σ_R²={sigma_R2:.3f}, r0={total_r0*1000:.2f} mm")

    # Helper: safe beam waist fallback
    def safe_beam_waist(beam_obj, z, w0_default):
        try:
            if beam_obj is None:
                zR = np.pi * (w0_default**2) / config['wavelength']
                return w0_default * np.sqrt(1 + (z / zR)**2)
            return beam_obj.beam_waist(z)
        except Exception:
            zR = np.pi * (w0_default**2) / config['wavelength']
            return w0_default * np.sqrt(1 + (z / zR)**2)

    # --- 3. CHANNEL PROPAGATION ---
    if config['verbose']:
        print("\n[3] Propagation Through Turbulence & Attenuation")

    all_metrics = {'ber': [], 'cond_H': [], 'strehl': [], 'scint': [], 'path_loss_db': []}
    rx_sequences = []

    for ens in range(config['n_ensembles']):
        if config['verbose']:
            print(f"   Ensemble {ens+1}/{config['n_ensembles']}")

        E_rx_ens = np.zeros((n_symbols, config['N_grid'], config['N_grid']), dtype=complex)

        grid_info = frame.grid_info
        if grid_info is None or 'R' not in grid_info:
            first_beam = None
            for b in tx.lg_beams.values():
                if b is not None:
                    first_beam = b
                    break
            w_L = safe_beam_waist(first_beam, config['distance'], config['w0'])
            D = float(config['oversampling']) * 6.0 * float(w_L)
            delta = D / float(config['N_grid'])
            x = np.linspace(-D/2, D/2, config['N_grid'])
            X, Y = np.meshgrid(x, x, indexing='ij')
            R, PHI = np.sqrt(X**2 + Y**2), np.arctan2(Y, X)
            grid_info = {'x': x, 'y': x, 'X': X, 'Y': Y, 'R': R, 'PHI': PHI, 'delta': delta}

        for t in range(n_symbols):
            E_mux_t = np.zeros((config['N_grid'], config['N_grid']), dtype=complex)
            for mode_key in config['spatial_modes']:
                tx_mode = tx_signals.get(mode_key)
                if tx_mode is None:
                    continue
                symbol = tx_mode['symbols'][t] if t < tx_mode['n_symbols'] else 0.0 + 0.0j
                beam = tx_mode.get('beam')
                if beam is not None:
                    try:
                        E_base = beam.generate_beam_field(grid_info['R'], grid_info['PHI'], 0.0)
                    except Exception:
                        E_base = None
                else:
                    E_base = None
                if E_base is None:
                    p, l = mode_key
                    w_eff = safe_beam_waist(beam, 0.0, config['w0'])
                    amp = np.exp(-(grid_info['R']**2)/(w_eff**2))
                    phase = np.exp(1j * l * grid_info['PHI'])
                    E_base = amp * phase
                E_mux_t += E_base * symbol

            p_mux = np.sum(np.abs(E_mux_t)**2) * (grid_info['delta']**2)
            if p_mux > 0:
                E_mux_t /= np.sqrt(p_mux / config['P_tx_watts'])
            if sigma_R2 > 0:
                result = apply_multi_layer_turbulence(
                    E_mux_t, beam_example, layers, config['distance'],
                    N=config['N_grid'], oversampling=config['oversampling'],
                    L0=config['L0'], l0=config['l0']
                )
                E_rx_ens[t] = result['final_field']
                strehl = np.max(np.abs(result['final_field'])**2) / np.max(np.abs(result['pristine_field'])**2)
                all_metrics['strehl'].append(strehl)
            else:
                E_rx_ens[t] = E_mux_t
                all_metrics['strehl'].append(1.0)
                all_metrics['scint'].append(0.0)

        # --- Attenuation ---
        L_geo_list, L_atm_list = [], []
        for mode in config['spatial_modes']:
            beam_obj = tx.lg_beams.get(mode, beam_example)
            try:
                geo_res = calculate_geometric_loss(beam_obj, config['distance'], config['receiver_radius'])
                L_geo_db = float(geo_res[0]) if isinstance(geo_res, tuple) else float(geo_res)
            except Exception:
                L_geo_db = 0.0
            try:
                if hasattr(beam_obj, 'calculate_path_loss'):
                    res = beam_obj.calculate_path_loss(config['distance'], config['receiver_radius'],
                                                       P_tx=config['P_tx_watts'], weather='clear', use_kim_model=True)
                    L_atm_db = res.get('L_atm_dB', 0.0)
                else:
                    atm = calculate_path_loss(beam_obj, config['distance'], rx_radius=config['receiver_radius'],
                                              Cn2=config['ground_Cn2'])
                    L_atm_db = atm if np.isscalar(atm) else 0.0
            except Exception:
                L_atm_db = 0.0
            L_geo_list.append(L_geo_db)
            L_atm_list.append(L_atm_db)
        L_geo_db = np.mean(L_geo_list)
        L_atm_db = np.mean(L_atm_list)
        L_total_db = L_geo_db + L_atm_db
        scale = 10.0 ** (-L_total_db / 20.0)
        E_rx_ens *= scale
        all_metrics['path_loss_db'].append(L_total_db)
        rx_sequences.append(E_rx_ens)

    # --- 4. RECEIVER ---
    if config['verbose']:
        print("\n[4] Receiver Stage")

    eq_method = 'mmse' if (config['eq_method'] == 'auto' and config['ground_Cn2'] > 1e-16) else config['eq_method']
    if args.zf:
        eq_method = 'zf'

    rx = FSORx(
        spatial_modes=config['spatial_modes'],
        wavelength=config['wavelength'],
        w0=config['w0'],
        z_distance=config['distance'],
        pilot_handler=tx.pilot_handler,
        ldpc_instance=tx.ldpc,
        eq_method=eq_method,
        receiver_radius=config['receiver_radius']
    )

    all_bers, all_cond_H = [], []
    for ens_idx, E_rx_ens in enumerate(rx_sequences):
        if config['verbose']:
            print(f"   RX Ensemble {ens_idx+1}")
        E_rx_list = [E_rx_ens[t] for t in range(n_symbols)]
        decoded_bits, metrics = rx.receive_sequence(
            E_rx_list, grid_info, tx_signals, original_data_bits=data_bits, verbose=config['verbose']
        )
        all_bers.append(metrics.get('ber', np.nan))
        H_est = metrics.get('H_est')
        cond_H = np.linalg.cond(H_est) if H_est is not None else np.nan
        all_cond_H.append(cond_H)
        if config['verbose']:
            print(f"      BER={metrics['ber']:.2e}, cond(H)={cond_H:.1f}")

    # --- Aggregate Metrics ---
    final_metrics = {
        'mean_ber': np.nanmean(all_bers),
        'std_ber': np.nanstd(all_bers),
        'mean_cond_H': np.nanmean(all_cond_H),
        'mean_strehl': np.nanmean(all_metrics['strehl']),
        'mean_path_loss_db': np.nanmean(all_metrics['path_loss_db']),
        'n_ensembles': config['n_ensembles'],
        'r0_mm': total_r0 * 1000,
        'sigma_R2': sigma_R2,
    }

    elapsed = (datetime.now() - start_time).total_seconds()
    if config['verbose']:
        print("\n" + "="*80)
        print(f"Pipeline Complete in {elapsed:.1f}s")
        print(f"Mean BER = {final_metrics['mean_ber']:.3e} ± {final_metrics['std_ber']:.3e}")
        print(f"Mean Strehl = {final_metrics['mean_strehl']:.3f}")
        print(f"Mean cond(H) = {final_metrics['mean_cond_H']:.1f}")
        print("="*80)

    # --- 5. Plotting ---
    if config['plot']:
        fig, axs = plt.subplots(2, 2, figsize=(12, 10))
        axs[0,0].plot(all_bers, 'o-')
        axs[0,0].set_yscale('log'); axs[0,0].set_title('BER per Ensemble')
        axs[0,0].set_xlabel('Ensemble'); axs[0,0].set_ylabel('BER')

        axs[0,1].hist(all_cond_H, bins=5)
        axs[0,1].set_title(f'Condition Number (mean {final_metrics["mean_cond_H"]:.1f})')

        axs[1,0].bar(['Strehl', 'Path Loss (dB)'],
                     [final_metrics['mean_strehl'], final_metrics['mean_path_loss_db']])
        axs[1,0].set_title('Channel Effects')

        if len(rx_sequences) > 0:
            last_rx = rx_sequences[-1]
            snapshot = last_rx[0]
            tx_beams_map = {k: tx_signals[k].get('beam') for k in tx_signals}
            sym_snapshot = rx.demux.project_field(snapshot, grid_info,
                                                 receiver_radius=config['receiver_radius'],
                                                 tx_beams_map=tx_beams_map)
            s0 = sym_snapshot.get(config['spatial_modes'][0], 0.0 + 0.0j)
            axs[1,1].plot(np.real(s0), np.imag(s0), 'o')
            axs[1,1].set_title('Sample Constellation (Mode 0)')
            axs[1,1].axis('equal'); axs[1,1].grid(True)
        else:
            axs[1,1].text(0.5, 0.5, 'No RX Seq', ha='center')

        plt.suptitle(f"FSO-OAM Pipeline Results (L={config['distance']} m, Cn²={config['ground_Cn2']:.1e})")
        plt.tight_layout()
        if config['save_plots']:
            os.makedirs("plots", exist_ok=True)
            plt.savefig(f"plots/pipeline_{datetime.now().strftime('%Y%m%d_%H%M')}.png", dpi=150, bbox_inches='tight')
        plt.show()

    return final_metrics

# ===========================================================
# MAIN
# ===========================================================
if __name__ == "__main__":
    metrics = run_pipeline(config)
    print("\nFinal Metrics:", metrics)


FSO-OAM Pipeline Simulation (Literature-Aligned)

[1] Transmitter Initialization


AttributeError: property 'rate' of 'PyLDPCWrapper' object has no setter